# PROJET 10 DATA ANALYST

# OBJECTIF DE CE NOTEBOOK

Pour l'Organisation Nationale de lutte Contre le Faux-Monnayage (ONCFM), nous devons produire :

- Une analyse descriptive des données, notamment la répartition des dimensions des billets, le nombre de vrais / faux billets, etc.
- Une détection automatisée des faux billets à partir des dimensions de ces derniers. Les méthodes à utiliser sont la régression logistique et k-means avec une matrice de confusion pour évaluer les performances des modèles. Une fois la phase d'entrainement et de test achevée, l'algorithme devra être capable de prédire si un billet est vrai ou faux.

Glossaire :
- diagonal : la diagonale du billet (en mm)
- height_left : la hauteur du billet (mesurée sur le côté gauche, en mm)
- height_right : la hauteur du billet (mesurée sur le côté droit, en mm)
- length : la longueur du billet (en mm)
- margin_low : la marge entre le bord inférieur du billet et l'image de celui-ci (en mm)
- margin_up : la marge entre le bord supérieur du billet et l'image de celui-ci (en mm)

## Etape 1 - Importation des librairies et chargement des fichiers

## 1.1 - Importation des librairies

In [ ]:
import joblib
#Importation des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import scipy.stats as ss
from scipy.cluster.hierarchy import dendrogram, linkage

import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Chargement de la librairie graphique
sns.set()

## 1.2 - Chargement du fichier

In [ ]:
#Importation du fichier population.csv en mettant l'index sur 'Zone'
billet = pd.read_csv('./Data_transformed/billets_compl.csv', sep=';')

In [ ]:
#Affichage des dimensions et de leurs types
display(billet.info())

In [ ]:
#Affichage d'un échantillon
display(billet.sample(10))

## Etape 2 - Split des données et Centrage, Reduction (Scaling)

## 2.1 - Split des données

In [ ]:
#Sélection des variables explicatives et de la variable à expliquer
billet_y = billet['is_genuine']
billet_X = billet.drop('is_genuine', axis=1)

## 2.2 - Centrage et Réduction

In [ ]:
#Instanciation du Scaler
std_scale = preprocessing.StandardScaler()

#Entrainement
std_scale.fit(billet_X)

In [ ]:
#Transformation
billet_X_scaled = std_scale.transform(billet_X)

In [ ]:
#Remettre les données centrées/réduites dans un dataframe
billet_X_scal = pd.DataFrame(billet_X_scaled, columns=billet_X.columns)

In [ ]:
#Pour vérifier que le centrage/réduction s'est bien passé
display(pd.DataFrame(billet_X_scal).describe().round(2).iloc[1:3:, : ])

In [ ]:
#Concaténation des données centrées/réduites avec la variable à expliquer
billet_scaled = pd.concat([billet_y, billet_X_scal], axis=1)

display(billet_scaled.sample(10))

## Etape 3 - Classification et clustering

## 3.1 - Train/Test Split

In [ ]:
y = billet_scaled['is_genuine']
X = billet_scaled.drop('is_genuine', axis=1)

In [ ]:
#Séparation des données en train et test avec stratification pour conserver la proportion de vrais/faux billets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
display(X_train)

## 3.2 - Dummy model

In [ ]:
#Calcul du score du modèle Dummy -> Null Accuracy
strategies = ['most_frequent', 'stratified', 'uniform']#, 'constant']
test_scores = []

for s in strategies:
    if s =='constant':
        dclf = DummyClassifier(strategy = s, random_state = 0, constant ='M')
    else:
        dclf = DummyClassifier(strategy = s, random_state = 0)
    dclf.fit(X_train, y_train)
    score = dclf.score(X_test, y_test)
    test_scores.append(score)

test_scores = [round(aa, 4) for aa in test_scores]
print("Meilleurs scores des Dummy models : ", test_scores)

## 3.3 - Classification (supervisée) : Logistic Regression

## 3.3.1 - Learning curve pour déterminer le nombre d'individus optimal par Kfold

In [ ]:
# TODO : faire la learning curve avec le train set et non le dataset complet

## 3.3.2 - Détermination du nombre de variables explicatives à retenir

RFECV ne fournissant pas les scores du Train set, nous allons utiliser la GridSearchCV pour tester les différentes combinaisons de variables explicatives et ainsi déterminer le nombre de variables explicatives à retenir.

In [ ]:
#Définition du dictionnaire des variables explicatives à tester
hyper_params = [{'n_features_to_select': list(range(1, 7))}]

#Instanciation du modèle et de la Recursive Feature Elimination (RFE)
lr = LogisticRegression(random_state=0)
rfe = RFE(lr)

#Instanciation de la GridSearchCV
model_cv = GridSearchCV(estimator = rfe,
                        param_grid = hyper_params,
                        scoring= 'accuracy',
                        cv = 5, #Par défaut StratifiedKFold quand l'estimateur est un classifieur
                        verbose = 1,
                        return_train_score=True)

#Fit de la GridSearchCV
model_cv.fit(X_train, y_train)

In [ ]:
#Résultats de la GridSearchCV
cv_results = pd.DataFrame(model_cv.cv_results_)

#Sélection des colonnes utiles
cols = [i for i in cv_results.columns if not i.startswith('split') and not i.endswith('time')]
cv_results = cv_results.loc[:, cols]

display(cv_results)

In [ ]:
#Affichage de la contribution des variables explicatives à la performance du modèle
plt.figure(figsize=(15,5))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])

plt.xlabel('Nombre de variables explicatives')
plt.ylabel('Accuracy')
plt.title('Contribution des variables explicatives à la performance du modèle')
plt.legend(['validation score', 'train score'], loc='upper left')
plt.show()

## 3.3.3 - Détermination des variables explicatives à retenir

In [ ]:
#Instanciation du modèle
lr = LogisticRegression(random_state=0)

#Instanciation de la Recursive Feature Elimination (RFE) et fit
rfecv_lr = RFECV(estimator=lr,
              cv=4,
              scoring='accuracy')

rfecv_lr.fit(X_train, y_train)

In [ ]:
#Sélection dans le train set des variables explicatives retenues par la RFE
X_train_rfecv = X_train.iloc[:, rfecv_lr.support_]

display(X_train_rfecv.sample(10))

## 3.3.4 - Détermination du paramètre de régularisation C

In [ ]:
#Définition du dictionnaire des variables explicatives à tester
hyper_params = [{'C': [0.1, 1, 10, 20, 30, 40, 50, 60, 70, 80]}]
                 #'penalty': ['l1', 'l2'],
                 #'max_iter': [100, 200, 300, 400, 500],
                 #'n_jobs': [-1]}]

#Instanciation du modèle et de la Recursive Feature Elimination (RFE)
lr = LogisticRegression(random_state=0)

#Instanciation de la GridSearchCV
model_cv = GridSearchCV(estimator = lr,
                        param_grid = hyper_params,
                        scoring= 'accuracy',
                        cv = 5, #Par défaut StratifiedKFold quand l'estimateur est un classifieur
                        verbose = 1,
                        n_jobs=-1,
                        return_train_score=True)

#Fit de la GridSearchCV
model_cv.fit(X_train_rfecv, y_train)

In [ ]:
#Résultats de la GridSearchCV
cv_results = pd.DataFrame(model_cv.cv_results_)

#Sélection des colonnes utiles
cols = [i for i in cv_results.columns if not i.startswith('split') and not i.endswith('time')]
cv_results = cv_results.loc[:, cols]

display(cv_results)
print("Meilleur Hyper-paramètre : ", model_cv.best_params_)

In [ ]:
#Affichage de la contribution des variables explicatives à la performance du modèle
plt.figure(figsize=(15,10))

plt.plot(cv_results["param_C"], cv_results["mean_test_score"])
plt.plot(cv_results["param_C"], cv_results["mean_train_score"])

plt.xlabel('Force de la Régularisation')
plt.ylabel('Accuracy')
plt.title('Efficacité de la Régularisation dans la performance du modèle')
plt.legend(['validation score', 'train score'], loc='lower right')
plt.show()

## 3.3.5 - Validation sur le Test set, Matrice de confusion  et F1 Score

In [ ]:
#Instanciation du modèle
lr = LogisticRegression(random_state=0, C=30)

#Fit du modèle
model_lr = lr.fit(X_train_rfecv, y_train)
print("Train set Accuracy : ", round(model_lr.score(X_train_rfecv, y_train), 4))

In [ ]:
#Fonction pour le calcul de l'accuracy score (prédiction inversée en cas d'apprentissage non supervisé avec Target inversée)
def acc(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    if accuracy < 0.5:
        #Inversion des prédictions en cas d'apprentissage non supervisé avec Target inversée
        y_pred = (~y_pred.astype(bool)).astype(int)
        accuracy = accuracy_score(y_true, y_pred)

    print("Accuracy : ",round(accuracy,4))
    return y_pred

In [ ]:
#Sélection dans le Test set des variables explicatives retenues par la RFECV
X_test_rfecv = X_test.iloc[:, rfecv_lr.support_]

y_pred_lr = model_lr.predict(X_test_rfecv)

#Calcul de l'accuracy score et inversion des prédictions en cas d'apprentissage non supervisé avec Target inversée
y_pred_lr = acc(y_test, y_pred_lr)

**Matrice de confusion et F1 score**

In [ ]:
#Matrice de confusion
conf_mat = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(conf_mat, annot=True, cbar=None, cmap='Reds', fmt='.0f')

plt.ylabel('Réel')
plt.xlabel('Prévision');
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_lr, digits=4))

## 3.4 - Clustering (non supervisé) : K-Means 

## 3.4.1 - Détermination des variables explicatives à retenir

Nous avons pu remarquer dans le Notebook précédent que les variables explicatives ne sont pas fortement corrélées entre elles (mis à part le coeff de Pearson entre 'margin_low' et 'length' = -0.67). Nous allons donc utiliser toutes les variables explicatives pour le K-Means.

## 3.4.2 - Validation sur le Test set, Matrice de confusion et F1 Score

In [ ]:
#Instanciation et entrainement de l'estimateur
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X_train)

In [ ]:
#Extraction des labels
y_pred_km_train = kmeans.labels_

In [ ]:
#Calcul de l'accuracy score et inversion des prédictions en cas d'apprentissage non supervisé avec Target inversée
y_pred_km_train = acc(y_train, y_pred_km_train)

In [ ]:
#Prédiction sur le test set
y_pred_km_test = kmeans.predict(X_test)

In [ ]:
#Calcul de l'accuracy score et inversion des prédictions en cas d'apprentissage non supervisé avec Target inversée
y_pred_km_test = acc(y_test, y_pred_km_test)

**Matrice de confusion et F1 score**

In [ ]:
#Matrice de confusion
conf_mat = confusion_matrix(y_test, y_pred_km_test)
sns.heatmap(conf_mat, annot=True, cbar=None, cmap='Reds', fmt='.0f')

plt.ylabel('Réel')
plt.xlabel('Prévision');
plt.show()

In [ ]:
#Calcul du F1 score
print(classification_report(y_test, y_pred_km_test, digits=4))

## 3.5 - Classification (supervisé) : K-Nearest Neighbors 
- Learning curve sur le nombre d'individus optimal pour le train set
- GridSearchCV pour trouver le nombre de voisins optimal sur le train set

## 3.5.1 - Détermination des variables explicatives à retenir

Nous avons pu remarquer dans le Notebook précédent que les variables explicatives ne sont pas fortement corrélées entre elles (mis à part le coeff de Pearson entre 'margin_low' et 'length' = -0.67). Nous allons donc utiliser toutes les variables explicatives pour le K-NN.

## 3.5.2 - Détermination du nombre de voisins optimal

In [ ]:
#Définition du dictionnaire des variables explicatives à tester
hyper_params = [{'n_neighbors': list(range(1, 51))}]

#Instanciation du modèle et de la Recursive Feature Elimination (RFE)
knn = KNeighborsClassifier()

#Instanciation de la GridSearchCV
model_cv = GridSearchCV(estimator = knn,
                        param_grid = hyper_params,
                        scoring= 'accuracy',
                        cv = 5, #Par défaut StratifiedKFold quand l'estimateur est un classifieur
                        verbose = 1,
                        n_jobs=-1,
                        return_train_score=True)

#Fit de la GridSearchCV
model_cv.fit(X_train, y_train)

In [ ]:
#Résultats de la GridSearchCV
cv_results = pd.DataFrame(model_cv.cv_results_)

#Sélection des colonnes utiles
cols = [i for i in cv_results.columns if not i.startswith('split') and not i.endswith('time')]
cv_results = cv_results.loc[:, cols]

display(cv_results)
print("Meilleur Hyper-paramètre : ", model_cv.best_params_)

In [ ]:
#Affichage de la contribution des variables explicatives à la performance du modèle
plt.figure(figsize=(15,10))

plt.plot(cv_results["param_n_neighbors"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_neighbors"], cv_results["mean_train_score"])

plt.xlabel('Nombre de voisins')
plt.ylabel('Accuracy')
plt.title('Influence du nombre de voisins dans la performance du modèle')
plt.legend(['validation score', 'train score'], loc='lower right')
plt.show()

## 3.5.3 - Validation sur le Test set, Matrice de confusion et F1 Score

In [ ]:
#Instanciation du modèle
knn = KNeighborsClassifier(n_neighbors=7, n_jobs=-1)

#Fit du modèle
model_knn = knn.fit(X_train, y_train)
print("Train set Accuracy : ", round(model_knn.score(X_train, y_train), 4))

In [ ]:
#Prévision sur le test set
y_pred_knn = model_knn.predict(X_test)

#Calcul de l'accuracy score et inversion des prédictions en cas d'apprentissage non supervisé avec Target inversée
y_pred_knn = acc(y_test, y_pred_knn)

**Matrice de confusion et F1 score**

In [ ]:
#Matrice de confusion
conf_mat = confusion_matrix(y_test, y_pred_knn)
sns.heatmap(conf_mat, annot=True, cbar=None, cmap='Reds', fmt='.0f')

plt.ylabel('Réel')
plt.xlabel('Prévision');
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_knn, digits=4))

## 3.6 - Création d'un Pipeline et export du binaire du modèle

En regardant le F1 score des faux billets des différents modèles, nous pouvons voir que le régression logistique est le modèle qui a le meilleur score. Nous allons donc créer un pipeline avec ce modèle et exporter le binaire du modèle.

In [ ]:
quant_cols = list(billet_X.iloc[:, rfecv_lr.support_])

quant_pipeline = Pipeline(steps=[
    ('scale', preprocessing.StandardScaler())
    ])

col_trans = ColumnTransformer(transformers=[
    ('quant_pipeline',quant_pipeline,quant_cols)],
    remainder='passthrough',
    n_jobs=-1)

logreg = LogisticRegression(random_state=0, C=30)

logreg_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', logreg)
    ])

display(logreg_pipeline)

In [ ]:
billet_X_pipe = billet_X[quant_cols].copy()

#Séparation des données en train et test avec stratification pour conserver la proportion de vrais/faux billets
X_train_pipe, X_test_pipe, y_train_pipe, y_test_pipe = train_test_split(billet_X_pipe, billet_y, test_size=0.2, random_state=0, stratify=y)
display(X_train_pipe)

In [ ]:
#Fit du pipeline
logreg_pipeline.fit(X_train_pipe, y_train_pipe)

score = logreg_pipeline.score(X_test_pipe, y_test_pipe)
print("Test set accuracy : ", score) # model accuracy

Nous retrouvons bien le même score que précédemment.

In [ ]:
#y_pred_pipe = logreg_pipeline.predict(X_test_pipe)

In [ ]:
# Save pipeline to file "pipe.joblib"
joblib.dump(logreg_pipeline,"./logreg_pipeline.joblib")

In [ ]:
lr_pipeline = joblib.load("./logreg_pipeline.joblib")
y_pred_pipe = lr_pipeline.predict(X_test_pipe)
display(y_pred_pipe)